# 编制音乐索引

捕捉音乐风格，建立索引，实现音乐的搜索引擎，就可以用来听歌识曲了，类似于 Shazam。

In [30]:
import random
random.seed(42)

import numpy as np
np.random.seed(42)

import tensorflow as tf
tf.random.set_seed(42)

这里做了个简单的数据集，包含我随便选的 131 首歌：

In [13]:
DIR = "/Volumes/shared/murecom/intro/idx-local-mp3/"

def myfile(name):
    return os.path.join(DIR, name)


import os
import pandas

pandas.read_csv(myfile("songs.csv"))

,类型（A专辑M合集）,曲目数量,名称,艺人,语种,风格(选取理由)
0,A,17,Revival,Eminem,英语,Hip-Hop (转型之作，有大量流行等现代元素 删了两首太短的Interlude)
1,A,12,死灰,邪恶少年EB,中文,Hip-Hop（OldSchool 删了 Intro）
2,A,8,One Last Kiss,宇多田光,日语,Pop (有多个版本Beautiful World 另有两首有Instrumental)
3,M,22,国语老歌随机合集,-,中文,流行 & 摇滚
4,M,16,欧美怀旧盗版碟,-,英文,流行 & 摇滚
5,A,14,奇跡の山,岸部真明,纯音乐,指弹
6,A,21,VIOLET EVERGARDEN：Automemories - Disc 1,Evan Call,纯音乐,原声
7,M,9,世纪乐典-古琴名曲,-,纯音乐,中国古代
8,M,12,古典乐随机合集,-,纯音乐,古典


把音乐提取出来：

In [14]:
MUSIC_ROOT = myfile("mp3s")

mp3s = []
for root, subdirs, files in os.walk(MUSIC_ROOT):
    for fn in files:
        if fn.endswith('.mp3'):
            mp3s.append(os.path.join(root, fn))
len(mp3s)

131

处理 MP3 文件，获取梅尔频谱图

In [21]:
import librosa
import numpy as np
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

def process_mp3(path):
    signal, sr = librosa.load(path,
                              res_type="kaiser_fast",
                              offset=30,
                              duration=30)
    melspec = librosa.feature.melspectrogram(signal, sr=sr).T[:1280, ]
    if len(melspec) != 1280:
        return None
    return {'path': path,
            'melspecs': np.asarray(np.split(melspec, 10))}

songs = [process_mp3(path) for path in tqdm(mp3s)]
songs = [song for song in songs if song]

len(songs)



  0%|          | 0/131 [00:00<?, ?it/s]

  1%|          | 1/131 [00:00<01:48,  1.20it/s]

  2%|▏         | 2/131 [00:01<01:25,  1.51it/s]

  2%|▏         | 3/131 [00:01<01:19,  1.62it/s]

  3%|▎         | 4/131 [00:02<01:14,  1.70it/s]

  4%|▍         | 5/131 [00:03<01:13,  1.71it/s]

  5%|▍         | 6/131 [00:03<01:15,  1.66it/s]

  5%|▌         | 7/131 [00:04<01:11,  1.74it/s]

  6%|▌         | 8/131 [00:04<01:08,  1.79it/s]

  7%|▋         | 9/131 [00:05<01:06,  1.83it/s]

  8%|▊         | 10/131 [00:05<01:05,  1.85it/s]

  8%|▊         | 11/131 [00:06<01:04,  1.87it/s]

  9%|▉         | 12/131 [00:06<01:03,  1.88it/s]

 10%|▉         | 13/131 [00:07<01:02,  1.89it/s]

 11%|█         | 14/131 [00:07<01:02,  1.87it/s]

 11%|█▏        | 15/131 [00:08<01:01,  1.89it/s]

 12%|█▏        | 16/131 [00:08<01:01,  1.88it/s]

 13%|█▎        | 17/131 [00:09<01:01,  1.84it/s]

 14%|█▎        | 18/131 [00:10<01:02,  1.81it/s]

 15%|█▍        | 19/131 [00:10<01:00,  1.85it/s]

 15%|█▌        |

131

为对每个 MP3 的所有频谱进行索引，我们可以把他们连在一起，方便一批完成

In [22]:
inputs = []

for song in songs:
    inputs.extend(song['melspecs'])
inputs = np.array(inputs)
inputs.shape

(1310, 128, 128)

用之前分类的那个训练好的模型，

In [23]:
from tensorflow.keras.models import load_model

cnn_model = load_model('./song_classify.h5')
cnn_model.summary()

 97%|█████████▋| 130/134 [33:40<01:02, 15.54s/it]
2022-01-07 16:02:15.591252: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 128, 128)]        0         
                                                                 
 conv1d_18 (Conv1D)          (None, 126, 64)           24640     
                                                                 
 batch_normalization_18 (Bat  (None, 126, 64)          256       
 chNormalization)                                                
                                                                 
 max_pooling1d_18 (MaxPoolin  (None, 63, 64)           0         
 g1D)                                                            
                                                                 
 conv1d_19 (Conv1D)          (None, 61, 128)           24704     
                                                                 
 batch_normalization_19 (Bat  (None, 61, 128)          512 

不要最后4层：

In [25]:
from tensorflow.keras import Model

vectorize_model = Model(inputs=cnn_model.input,
                        outputs=cnn_model.layers[-4].output)

把 inputs 送进去，跑出来一个歌曲(片段)的 256 维表示向量。

In [28]:
vectors = vectorize_model.predict(inputs)
vectors.shape

(1310, 256)

然后用一个最近邻模型就可以找出相似的歌曲：

In [29]:
from collections import Counter

from sklearn.neighbors import NearestNeighbors

nbrs = NearestNeighbors(n_neighbors=10, algorithm='ball_tree').fit(vectors)

def most_similar_songs(song_idx):
    distances, indices = nbrs.kneighbors(
        vectors[song_idx * 10: song_idx * 10 + 10])
    c = Counter()
    for row in indices:
        for idx in row[1:]:
            c[idx // 10] += 1
    return c.most_common()

随便指定首歌试一下：

In [46]:
def song_name(song_idx):
    return os.path.basename(songs[song_idx]['path'])

def print_similar_songs(song_idx, start=1, end=6):
    print("指定歌曲:", song_name(song_idx))
    # 跳过第一相似的：本身
    for idx, score in most_similar_songs(song_idx)[start:end]:
        print(f"[相似度{score}] {song_name(idx)}")

把曲目列出来方便找：

In [50]:
for s in range(len(songs)):
    print(f'{s} {song_name(s)}')

0 Evan Call - A Place to Call Home.mp3
1 Eminem - Remind Me.mp3
2 EB - 来一发.mp3
3 岸部眞明 - 暗の中で.mp3
4 EB - 活着.mp3
5 宇多田ヒカル - One Last Kiss (Instrumental).mp3
6 Eminem／Skylar Grey - Tragic Endings.mp3
7 Beyond - Amani.mp3
8 Returning Home.mp3
9 孙燕姿 - 遇见.mp3
10 Eagles - Hotel California.mp3
11 Scarborough Fair - Scarborough Fair.mp3
12 04 Longshuo Melody.mp3
13 Eminem／Alicia Keys - Like Home.mp3
14 岑宁儿 - 追光者.mp3
15 黑豹乐队 - 无地自容.mp3
16 宇多田ヒカル - One Last Kiss.mp3
17 宇多田ヒカル - Beautiful World (Da Capo Version) (Instrumental).mp3
18 岸部眞明 - 云の海.mp3
19 EB - 杀死差不多先生.mp3
20 Evan Call - A Bit of Sass.mp3
21 Claude Debussy - Reverie.mp3
22 岸部眞明 - 里山の夏.mp3
23 EB - F...,Love.mp3
24 Eminem - Castle.mp3
25 Bryan Adans - Everything I Do For You.mp3
26 Beyond - 喜欢你.mp3
27 Claudio Arrau - Waltz No. 10 in B minor, op. 69 no. 2.mp3
28 梁静茹 - 爱久见人心.mp3
29 Antonio Vivaldi - L'Estro Armonico No. 9 in D Major II. Larghetto.mp3
30 Lioned Richie - Say You Say Me.mp3
31 Eminem／Beyoncé - Walk On Water.mp3
32 I Just Call

In [54]:
print_similar_songs(17, start=0)

指定歌曲: 宇多田ヒカル - Beautiful World (Da Capo Version) (Instrumental).mp3
[相似度27] 宇多田ヒカル - Beautiful World (Da Capo Version).mp3
[相似度24] 宇多田ヒカル - Beautiful World (Da Capo Version) (Instrumental).mp3
[相似度6] Evan Call - Each Memory a Message.mp3
[相似度4] Evan Call - One Last Message.mp3
[相似度4] Richanel Max - Righte Here waiting.mp3
[相似度3] Evan Call - The Voice in My Heart.mp3


从 Instrumental 找到了有唱的版本。

In [58]:
print_similar_songs(50)

指定歌曲: Evan Call - Theme of Violet Evergarden.mp3
[相似度6] 齐秦 - 往事随风.mp3
[相似度5] Evan Call - An Admirable Doll.mp3
[相似度5] Evan Call - Ink to Paper.mp3
[相似度4] Evan Call - Rust.mp3
[相似度4] 贝多芬 - 致艾丽丝.mp3


混入了奇怪的东西。

In [59]:
print_similar_songs(65)

指定歌曲: 贝多芬 - Adagio Cantabile from Sonata Op.13 (Pathetique).mp3
[相似度10] Antonio Vivaldi - L'Estro Armonico No. 9 in D Major II. Larghetto.mp3
[相似度9] 贝多芬 - Adagio Cantabile from Sonata Op.13 (Pathetique).mp3
[相似度8] 贝多芬 - Symphony No. 9 in D minor.mp3
[相似度7] 贝多芬 - 致艾丽丝.mp3
[相似度7] Claudio Arrau - Waltz No. 10 in B minor, op. 69 no. 2.mp3


古典音乐找的非常好。

In [60]:
print_similar_songs(79)

指定歌曲: 岸部眞明 - 流れ行く云.mp3
[相似度12] 岸部眞明 - 流れ行く云.mp3
[相似度12] 岸部眞明 - 遥かなる富士.mp3
[相似度10] 岸部眞明 - 奇迹の山.mp3
[相似度6] 岸部眞明 - 云の海.mp3
[相似度6] 岸部眞明 - 里山の夏.mp3


指弹也能很好分辨

In [68]:
print_similar_songs(58)

指定歌曲: 03 Beautiful Evening Prelude.mp3
[相似度7] 06 Three Departures for Yangguan.mp3
[相似度7] Gong-Liang Cheng  - Drowning Sorrows.mp3
[相似度7] Evan Call - Rust.mp3
[相似度5] 03 Beautiful Evening Prelude.mp3
[相似度4] Evan Call - The Voice in My Heart.mp3


古琴也还好，但混了两首薇就很离谱，两首都是提琴，跟古琴哪像了。

In [64]:
print_similar_songs(128)

指定歌曲: Eminem - Offended.mp3
[相似度9] EB - 世界是你的.mp3
[相似度7] EB - 来一发.mp3
[相似度6] Beyond - 喜欢你.mp3
[相似度5] Eminem／Beyoncé - Walk On Water.mp3
[相似度4] 宇多田ヒカル - One Last Kiss (Instrumental).mp3


HipHop 也还行，可以跨越语种的找到，但会混一些其他的进去。

In [70]:
print_similar_songs(33)

指定歌曲: Beyond - 灰色轨迹.mp3
[相似度12] 齐秦 - 往事随风.mp3
[相似度10] Beyond - 灰色轨迹.mp3
[相似度8] Whan - CareLess Whisper.mp3
[相似度8] All Out Love Air Supply - All Out Love Air Supply.mp3
[相似度4] BeritrHiggins - Casablanca.mp3


流行也还行。

In [72]:
print_similar_songs(75, start=0)

指定歌曲: Beyond - 不再犹豫.mp3
[相似度8] Eminem - Untouchable.mp3
[相似度7] Beyond - 不再犹豫.mp3
[相似度6] EB - 你挡不住我.mp3
[相似度6] I Just Call To Say I Love You - I Just Call To Say I Love You.mp3
[相似度5] Eminem／Skylar Grey - Tragic Endings.mp3
[相似度5] Carpenters - Yesterday Once More.mp3


这个摇滚就不太好了，前面做分类摇滚也是分的很迷，这个模型就不太喜欢听摇滚吧。